In [ ]:
#def callback(lcl, glb):
    # stop training if reward exceeds 199
#    is_solved = lcl['t'] > 100 and sum(lcl['episode_rewards'][-101:-1]) / 100 >= 199
#    return is_solved

In [1]:
from baselines import deepq
from baselines.common import set_global_seeds
from baselines import bench
import argparse
from baselines import logger
from baselines.common.atari_wrappers import make_atari

/home/jupyter/anaconda3/envs/GVGAI_GYM/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import gym

from baselines import deepq

from IPython.core.debugger import Tracer

import gym_gvgai
#env = gym.make('aliens-gvgai-v0')


def main():
    logger.configure()
    set_global_seeds(0)
    #env = make_atari('BreakoutNoFrameskip-v4')
    #env = gym.make("BreakoutNoFrameskip-v4")
    env = gym.make('aliens-gvgai-v0')
    env = bench.Monitor(env, logger.get_dir())
    #Tracer()()
    #env = deepq.wrap_atari_dqn(env)
    # Enabling layer_norm here is import for parameter space noise!
    model = deepq.models.cnn_to_mlp(
        convs=[(32, 8, 4), (64, 4, 2), (64, 3, 1)],
        hiddens=[256],
        dueling=bool(int(1)),
    )
    act = deepq.learn(
        env,
        q_func=model,
        lr=1e-3,
        max_timesteps=100000,
        buffer_size=50000,
        exploration_fraction=0.1,
        exploration_final_eps=0.1,
        print_freq=10,
        #callback=callback
    )
    #print("Saving model to mountaincar_model.pkl")
    #act.save("mountaincar_model.pkl")


if __name__ == '__main__':
    main()

Logging to /tmp/openai-2018-03-02-22-24-39-876185
GVGAI_Env - Version 0.0.1
Connecting to host 127.0.0.1 at port 8080 ...
Client connected to server [OK]
GET SCREEN DIMENSIONS!!! (FIX: Right now just 110x300x3)
Instructions for updating:
keep_dims is deprecated, use keepdims instead


/home/jupyter/anaconda3/envs/GVGAI_GYM/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/jupyter/anaconda3/envs/GVGAI_GYM/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--------------------------------------
| % time spent exploring  | 45       |
| episodes                | 10       |
| mean 100 episode reward | 2.1e+04  |
| steps                   | 6004     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 11       |
| episodes                | 20       |
| mean 100 episode reward | 1.28e+04 |
| steps                   | 9797     |
--------------------------------------


ERROR:root:
Traceback (most recent call last):
  File "/home/jupyter/Notebooks/ruben/gvgai-gym_2/gym_gvgai/envs/gvgai/clients/GVGAI-PythonClient/src/utils/IOSocket.py", line 64, in readLine
    msg = self.recv_end()
  File "/home/jupyter/Notebooks/ruben/gvgai-gym_2/gym_gvgai/envs/gvgai/clients/GVGAI-PythonClient/src/utils/IOSocket.py", line 81, in recv_end
    total_data.append(data)
MemoryError


Read from server [FAILED]


Traceback (most recent call last):
  File "/home/jupyter/Notebooks/ruben/gvgai-gym_2/gym_gvgai/envs/gvgai/clients/GVGAI-PythonClient/src/utils/IOSocket.py", line 64, in readLine
    msg = self.recv_end()
  File "/home/jupyter/Notebooks/ruben/gvgai-gym_2/gym_gvgai/envs/gvgai/clients/GVGAI-PythonClient/src/utils/IOSocket.py", line 81, in recv_end
    total_data.append(data)
MemoryError


SystemExit: 

/home/jupyter/anaconda3/envs/GVGAI_GYM/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
env.unwrapped.ale.lives()

In [ ]:
def main():
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--env', help='environment ID', default='BreakoutNoFrameskip-v4')
    parser.add_argument('--seed', help='RNG seed', type=int, default=0)
    parser.add_argument('--prioritized', type=int, default=1)
    parser.add_argument('--dueling', type=int, default=1)
    parser.add_argument('--num-timesteps', type=int, default=int(10e6))
    args = parser.parse_args()
    logger.configure()
    set_global_seeds(args.seed)
    env = make_atari(args.env)
    env = bench.Monitor(env, logger.get_dir())
    env = deepq.wrap_atari_dqn(env)
    model = deepq.models.cnn_to_mlp(
        convs=[(32, 8, 4), (64, 4, 2), (64, 3, 1)],
        hiddens=[256],
        dueling=bool(args.dueling),
    )
    act = deepq.learn(
        env,
        q_func=model,
        lr=1e-4,
        max_timesteps=args.num_timesteps,
        buffer_size=10000,
        exploration_fraction=0.1,
        exploration_final_eps=0.01,
        train_freq=4,
        learning_starts=10000,
        target_network_update_freq=1000,
        gamma=0.99,
        prioritized_replay=bool(args.prioritized)
    )
    # act.save("pong_model.pkl") XXX
    env.close()


if __name__ == '__main__':
    main()